In [1]:
import onnx
import onnxruntime as ort
import numpy as np

In [2]:
print(ort.__version__)

1.18.0


In [3]:
def inspect_model(model_path: str, n_track_features: int, none_input: bool = False):
    model = onnx.load(model_path)
    onnx.checker.check_model(model)
    onnx.helper.printable_graph(model.graph)

    sess_opts = ort.SessionOptions()
    sess_opts.enable_mem_reuse = False
    ort_sess = ort.InferenceSession(model_path, sess_opts=sess_opts)
    input_names = [x.name for x in ort_sess.get_inputs()]
    input_shapes = [x.shape for x in ort_sess.get_inputs()]
    output_names = [x.name for x in ort_sess.get_outputs()]
    output_shapes = [x.shape for x in ort_sess.get_outputs()]
    print("Input names: ", input_names, "with shapes: ", input_shapes)
    print("Output names: ", output_names, "with shapes: ", output_shapes)


    jet_features = np.array([[85507.8, -3.05748]], dtype=np.float32)
    track_features = np.array([]*n_track_features, dtype=np.float32) \
        if none_input \
        else np.array([0.]*n_track_features, dtype=np.float32)
    track_features = track_features.reshape(-1, n_track_features)
    print("jet feature shape:", jet_features.shape)
    print("track feature shape:", track_features.shape)

    results = ort_sess.run(output_names, {input_names[0]: jet_features, input_names[1]: track_features})
    return results


In [4]:
m1_config = ["/cvmfs/atlas.cern.ch/repo/sw/database/GroupData/dev/BTagging/20230307/gn2v00/antiktvr30rmax4rmin02track/network.onnx", 21]
m2_config = ["/cvmfs/atlas.cern.ch/repo/sw/database/GroupData/dev/BTagging/20231205/GN2v01/antikt4empflow/network_fold0.onnx", 19]

In [5]:
inspect_model(*m1_config)

Input names:  ['jet_features', 'track_features'] with shapes:  [[1, 2], ['n_tracks', 21]]
Output names:  ['pu', 'pc', 'pb'] with shapes:  [[1], [1], [1]]
jet feature shape: (1, 2)
track feature shape: (1, 21)


[array([0.8281662], dtype=float32),
 array([0.10067604], dtype=float32),
 array([0.07115781], dtype=float32)]

In [6]:
inspect_model(*m1_config, none_input=True)

Input names:  ['jet_features', 'track_features'] with shapes:  [[1, 2], ['n_tracks', 21]]
Output names:  ['pu', 'pc', 'pb'] with shapes:  [[1], [1], [1]]
jet feature shape: (1, 2)
track feature shape: (0, 21)


[array([0.6114677], dtype=float32),
 array([0.26959956], dtype=float32),
 array([0.11893278], dtype=float32)]

In [7]:
inspect_model(*m2_config)

Input names:  ['jet_features', 'track_features'] with shapes:  [[1, 2], ['n_tracks', 19]]
Output names:  ['GN2v01_pb', 'GN2v01_pc', 'GN2v01_pu', 'GN2v01_ptau', 'GN2v01_TrackOrigin', 'GN2v01_VertexIndex'] with shapes:  [[], [], [], [], ['n_tracks'], ['n_tracks']]
jet feature shape: (1, 2)
track feature shape: (1, 19)


[array(0.00410247, dtype=float32),
 array(0.07487475, dtype=float32),
 array(0.5267351, dtype=float32),
 array(0.39428762, dtype=float32),
 array([6], dtype=int8),
 array([0], dtype=int8)]

In [8]:
inspect_model(*m2_config, none_input=True)

Input names:  ['jet_features', 'track_features'] with shapes:  [[1, 2], ['n_tracks', 19]]
Output names:  ['GN2v01_pb', 'GN2v01_pc', 'GN2v01_pu', 'GN2v01_ptau', 'GN2v01_TrackOrigin', 'GN2v01_VertexIndex'] with shapes:  [[], [], [], [], ['n_tracks'], ['n_tracks']]
jet feature shape: (1, 2)
track feature shape: (0, 19)


2024-06-27 11:00:11.839008196 [E:onnxruntime:, sequential_executor.cc:516 ExecuteKernel] Non-zero status code returned while running Shape node. Name:'/Shape' Status Message: /onnxruntime_src/onnxruntime/core/framework/op_kernel.cc:83 virtual OrtValue* onnxruntime::OpKernelContext::OutputMLValue(int, const onnxruntime::TensorShape&) status.IsOK() was false. Shape mismatch attempting to re-use buffer. {2} != {3}. Validate usage of dim_value (values should be > 0) and dim_param (all values with the same string should equate to the same size) in shapes in the model.



RuntimeException: [ONNXRuntimeError] : 6 : RUNTIME_EXCEPTION : Non-zero status code returned while running Shape node. Name:'/Shape' Status Message: /onnxruntime_src/onnxruntime/core/framework/op_kernel.cc:83 virtual OrtValue* onnxruntime::OpKernelContext::OutputMLValue(int, const onnxruntime::TensorShape&) status.IsOK() was false. Shape mismatch attempting to re-use buffer. {2} != {3}. Validate usage of dim_value (values should be > 0) and dim_param (all values with the same string should equate to the same size) in shapes in the model.


In [ ]:
# numbers are taken from: 
# https://gitlab.cern.ch/atlas/athena/-/merge_requests/69721

# jet_features = np.array([[11516.2, 1.0935]], dtype=np.float32)

# track_features = np.array([-0.113795, 0.173364, -0.230979, -0.233828, 0.000914455, 1.1331, 1.3538, 0.00291881, 0.00241196, 1.61412e-05, 4, 8, 1, 1, 0, 0, 0, 0, 0, 0, 0, -0.126863, 0.00371974, 0.17269, 0.135184, -0.000857426, 0.904411, -0.0226558, 0.00404071, 0.00238144, 1.47576e-05, 4, 7, 1, 1, 0, 0, 0, 0, 0, 0, 1], dtype=np.float32)
# track_features = track_features.reshape(-1, n_track_features)

# print("jet feature shape:", jet_features.shape)
# print("track feature shape:", track_features.shape)

# pu, pc, pb = ort_sess.run(output_names, {input_names[0]: jet_features, input_names[1]: track_features})

# print(f"pu: {pu}")
# print(f"pc: {pc}")
# print(f"pb: {pb}")